In [98]:
import gym
import random
import torch
import torch.nn as nn
import numpy as np

In [99]:
env = gym.make('CartPole-v0')

In [100]:
class SequentialDQNModel(nn.Module):
	def __init__(self):
		super(SequentialDQNModel, self).__init__()

		self.logits = nn.Sequential(
			nn.Flatten(),
			nn.Linear(4, 24),
			nn.Tanh(),
			nn.Linear(24, 24),
			nn.ReLU(),
			nn.Linear(4, 2)
		)
		
		self.memory = []
	
	def remember(self, state, action ,reward, next_state, done):
		self.memory.append((state, action, reward, next_state, done))
	
	def f(self, state):
		return torch.softmax(self.logits(state), dim=1)
	
	def loss(self, state, y):
		return nn.functional.cross_entropy(self.logits(state), y.argmax(1))
	


In [101]:
model = SequentialDQNModel()

lr = 0.3
gamma = 0.95
epsilon = 0.1

In [102]:
for i in range(1, 5000):
	state = env.reset()

	epochs, reward = 0,0
	action = -1
	done  = False
	while not done:
		state = torch.tensor(state).reshape(1,-1)
		if(random.uniform(0, 1) < epsilon):
			action = env.action_space.sample()
		else:
			action = np.argmax(model.f(state))
		
		next_state, reward, done, info = env.step(action)
		
		# old_value = q_table[state + (action, )]
		# next_max = np.max(q_table[next_state])
		# new_value = (1-lr)*old_value + lr * (reward + gamma * next_max )
		# q_table[state + (action, )] = new_value
		
		state = next_state
		epochs +=1
	
	

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x24 and 4x2)